In [ ]:
!pip install deepface pycryptodome


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.8 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=c44247feec978442ab85a6a6f899e473d65fb2d29511606738fc7a150429d729
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [ ]:
from google.colab import files

print("📷 Upload your face image for encryption")
ref_image = files.upload()

print("📄 Upload the file you want to encrypt")
file_to_encrypt = files.upload()


📷 Upload your face image for encryption


Saving priyanuj.jpg to priyanuj.jpg
📄 Upload the file you want to encrypt


Saving secret.txt.txt to secret.txt.txt


In [ ]:
from deepface import DeepFace
from Crypto.Cipher import AES
from Crypto.Protocol.KDF import PBKDF2
import base64
import hashlib
import os

ref_img_filename = list(ref_image.keys())[0]
file_name = list(file_to_encrypt.keys())[0]

# Generate face vector using DeepFace
def get_face_embedding(img_path):
    obj = DeepFace.represent(img_path=img_path, model_name="Facenet", enforce_detection=True)[0]
    return obj["embedding"]

# Derive AES key from face embedding
def get_key_from_embedding(embedding):
    encoding_bytes = str(embedding).encode()
    salt = b'unique_salt'
    return PBKDF2(encoding_bytes, salt, dkLen=32)

# Encrypt file
def encrypt_file(file_path, key):
    with open(file_path, 'rb') as f:
        plaintext = f.read()

    cipher = AES.new(key, AES.MODE_EAX)
    ciphertext, tag = cipher.encrypt_and_digest(plaintext)
    encrypted = base64.b64encode(cipher.nonce + tag + ciphertext)

    with open("encrypted_file.bin", "wb") as f:
        f.write(encrypted)

    print("✅ File encrypted and saved as encrypted_file.bin")

# Process
embedding = get_face_embedding(ref_img_filename)
aes_key = get_key_from_embedding(embedding)
encrypt_file(file_name, aes_key)


25-04-08 08:12:48 - Directory /root/.deepface has been created
25-04-08 08:12:48 - Directory /root/.deepface/weights has been created
25-04-08 08:12:52 - facenet_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet_weights.h5
To: /root/.deepface/weights/facenet_weights.h5
100%|██████████| 92.2M/92.2M [00:02<00:00, 44.9MB/s]


✅ File encrypted and saved as encrypted_file.bin


In [ ]:
print("🧑 Upload face image for decryption (must match original)")
verify_face = files.upload()
verify_img_filename = list(verify_face.keys())[0]

# Verify face
is_same = DeepFace.verify(img1_path=ref_img_filename, img2_path=verify_img_filename, model_name="Facenet")["verified"]

if is_same:
    print("✅ Face matched. Decrypting...")

    key = get_key_from_embedding(get_face_embedding(verify_img_filename))

    with open("encrypted_file.bin", "rb") as f:
        data = base64.b64decode(f.read())

    nonce = data[:16]
    tag = data[16:32]
    ciphertext = data[32:]

    cipher = AES.new(key, AES.MODE_EAX, nonce)
    try:
        decrypted = cipher.decrypt_and_verify(ciphertext, tag)
        with open("decrypted_file.txt", "wb") as f:
            f.write(decrypted)
        print("📄 File decrypted and saved as decrypted_file.txt")
    except Exception as e:
        print("❌ Decryption failed:", str(e))
else:
    print("🚫 Face did not match. Access denied.")


🧑 Upload face image for decryption (must match original)


Saving priyanuj.jpg to priyanuj (1).jpg
✅ Face matched. Decrypting...
📄 File decrypted and saved as decrypted_file.txt
